# LoRA Tuning

In [1]:
import pandas as pd

train_data = pd.read_csv('train_data.csv')
validation_data = pd.read_csv('validation_data.csv')
test_data = pd.read_csv('test_data.csv')

In [2]:
print(train_data.head(5))

   labels                                              input
0       3  Lots of fun stuff!: This box was stuffed full ...
1       4  Where is the link to subscribe for 8-13?: We l...
2       4  Worth it!: Loved everything in there, definite...
3       4                Educational!: My grandkids love it!
4       4  Great small company to support: I love these b...


In [3]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(validation_data)
test_dataset = Dataset.from_pandas(test_data)

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
def preprocess_function(examples):
    return tokenizer(examples['input'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8557 [00:00<?, ? examples/s]

Map:   0%|          | 0/2852 [00:00<?, ? examples/s]

Map:   0%|          | 0/2853 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification

num_labels = len(train_data['labels'].unique())
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=num_labels, problem_type="single_label_classification")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
print(num_labels)

5


In [7]:
import numpy as np
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
np.object = object
np.bool = bool
np.int = int
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results_lora',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': (predictions == labels).mean()}


In [8]:
from peft import LoftQConfig, LoraConfig, get_peft_model,TaskType

lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=8,lora_alpha=16, target_modules=["query", "value"], modules_to_save=["classifier"])
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 889,349 || all params: 135,793,162 || trainable%: 0.6549


In [9]:
    
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.213300,0.897361,0.654979
2,0.837400,0.793810,0.687237
3,0.768300,0.759687,0.701262
4,0.738600,0.735153,0.711080
5,0.740500,0.728307,0.716690
6,0.707600,0.721861,0.716690
7,0.699600,0.714057,0.715638
8,0.699400,0.713049,0.718093
9,0.718400,0.714970,0.718794
10,0.699100,0.714333,0.718443


TrainOutput(global_step=5350, training_loss=0.776167696658696, metrics={'train_runtime': 1262.0532, 'train_samples_per_second': 67.802, 'train_steps_per_second': 4.239, 'total_flos': 5687200886799360.0, 'train_loss': 0.776167696658696, 'epoch': 10.0})

In [11]:
results = trainer.evaluate(test_dataset)
print(results)

{'eval_loss': 0.6907777190208435, 'eval_accuracy': 0.7262530669470733, 'eval_runtime': 16.806, 'eval_samples_per_second': 169.761, 'eval_steps_per_second': 10.651, 'epoch': 10.0}


In [12]:
model_path = "./lora_8"
trainer.save_model(model_path)